In [6]:
using Optim
using ForwardDiff # Mostly relies on package DualNumbers -> check in more detail later



BK_cost (generic function with 1 method)

In [7]:
# Subfunctions used for MBAM
function log_deriv_wrapper(f::Function, x::Vector; log_specific=ones(size(x)) )
    y = copy(x)
    # Assuming some of the input parameters (x) are in log space, passes the original (exponentiated) values on to the function f
    if sum(log_specific==1)==prod(size(x)) # all the parameters are in log space
        y = exp(y)
    else
        for i1 = 1:size(log_specific)[1]
            if log_specific[i1]==1
                y[i1]=exp(y[i1])
            end
        end
    end
    
    return f(y)
end

function to_log(x::Vector; log_specific=ones(size(x)))
    y = copy(x) # To avoid passing by reference!!!
    
    if sum(log_specific==1)==prod(size(x)) # all the parameters are in log space
        y = log(y)
    else
        for i1 = 1:size(log_specific)[1]
            if log_specific[i1]==1
                y[i1]=log(y[i1])
            end
        end
    end
    
    return y
end

to_log (generic function with 1 method)

In [11]:
function MBAM(f_costs::Function, x0::Vector, f_reds::Function; model_iters=0, boundary_iters=1000, stepsize=0.1, log_specific=ones(size(x0)))
    x_red = to_log(x0, log_specific=log_specific);
    x = x_red;
    V = x_red;
    D = x_red;
    
    for m1 = model_iters # Outer loop, reduces the number of parameters in the model each step
        f_cost = x1 -> f_costs(x1, model_id=m1); # Current cost function
        g_cost = x1 -> log_deriv_wrapper(f_cost, x1); # Define which parameters we're taking in log space ([default: all])
        
        x = x_red;
        for i1 = 1:boundary_iters # Inner loop, finding the boundary
            #result = HessianResult(x)
            #ForwardDiff.hessian(result, g_cost, x);
            
            H = ForwardDiff.hessian(g_cost, x)
            (D, V) = eig(H)
        
            x = x - V[:,sortperm(D)[1]]*stepsize
            #print("\n")
            #print([m1, i1, sort(D)[1], g_cost(x)])
        end
        
        x_red = f_reds(x, model_id=m1+1, in_log=1); # Transformed representation
    end
    
    f_cost = x -> f_costs(x, model_id=model_iters[end]); # Current cost function
    g_cost = x -> log_deriv_wrapper(f_cost, x, log_specific=log_specific); # Define which parameters we're taking in log space ([default: all])
    #g_cost(x)
    #print((x, V, x_red, g_cost(x)))
    return x, g_cost(x), V, D, x_red
end

MBAM (generic function with 1 method)